In [ ]:
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
from skimage.measure import label

from topostats.filters import Filters
from topostats.io import LoadScans
from topostats.plottingfuncs import Colormap

colormap = Colormap()
cmap = colormap.get_cmap()

In [ ]:
file_path = Path("/Users/sylvi/Downloads/20230818_p4cat_SC_mgni_3ng.0_00022.spm")
assert file_path.exists()

loadscans = LoadScans(img_paths=[file_path], channel="Height")
loadscans.get_data()

data = loadscans.img_dict
print(data.keys())
image_data = data[list(data.keys())[0]]
print(image_data.keys())
image_raw = image_data["image_original"]
p2nm = image_data["pixel_to_nm_scaling"]
print(f"{p2nm=}")
print(f"Shape: {image_raw.shape}")

plt.imshow(image_raw, cmap=cmap)
plt.show()

# Flatten the image
filters = Filters(
    image=image_raw,
    filename="test",
    pixel_to_nm_scaling=p2nm,
    row_alignment_quantile=0.5,
    threshold_method="std_dev",
    threshold_std_dev={"above": 1.0, "below": 10.0},
    gaussian_size=1.01,
    gaussian_mode="nearest",
    remove_scars={
        "run": False,
    },
)
filters.filter_image()
flattened_image = filters.images["gaussian_filtered"]
plt.imshow(flattened_image, cmap=cmap)
plt.show()

# Get the size of the image in bytes
size_bytes = flattened_image.nbytes
print(f"{size_bytes=}")
# Convert to np.float16 to see if the size is smaller
flattened_image_16 = flattened_image.astype(np.float16)
size_bytes_16 = flattened_image_16.nbytes
print(f"{size_bytes_16=}")

print(f"ratio: {size_bytes_16/size_bytes}")

# print a number from the 16 bit image to see what the precision is
print(f"16 bit image: {flattened_image_16[0,0]}")
print(f"32 bit image: {flattened_image[0,0]}")

In [ ]:
# Grab a crop of the molecule
crop_size_x = 330
crop_size_y = 200
crop_x = 135
crop_y = 150
crop = flattened_image_16[crop_y : crop_y + crop_size_y, crop_x : crop_x + crop_size_x]
plt.imshow(crop, cmap=cmap)
plt.show()

In [ ]:
# Create a spliced image
spliced_image = np.zeros((400, 400))
spliced_image[0:crop_size_y, 0:crop_size_x] = crop
rotated_crop = np.rot90(crop, k=2)
plt.imshow(rotated_crop, cmap=cmap)
plt.show()

spliced_image[spliced_image.shape[0] - crop_size_y :, spliced_image.shape[1] - crop_size_x :] = rotated_crop

plt.imshow(spliced_image, cmap=cmap)
plt.show()

In [ ]:
# Convert spliced image to 16 bit
spliced_image_16 = spliced_image.astype(np.float16)
np.save("./tests/resources/example_catenanes.npy", spliced_image_16)

In [ ]:
# Create grain mask for them
grain_mask = (spliced_image_16 > 1.2).astype(np.bool_)
# label the grains
labelled_grain_mask = label(grain_mask)
plt.imshow(labelled_grain_mask)
plt.show()

# Save the grain mask
np.save("./tests/resources/example_catenanes_labelled_grain_mask_thresholded.npy", labelled_grain_mask)